In [2]:
import sys
import os
import speech_recognition as sr
from pydub import AudioSegment
import pandas as pd
from pathlib import Path
from sklearn.metrics import jaccard_score
import re
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="Key/SpeechReg-2b5d8171274a.json"

# Example Speech to text

In [4]:
from google.cloud import speech

audio = speech.RecognitionAudio(uri="gs://voicedata_speech/Voice/conversation 2.wav")
config = speech.RecognitionConfig(
            sample_rate_hertz=16000,
            language_code="en-US",
            enable_automatic_punctuation=True,
            enable_word_time_offsets=True,
            model="phone_call")

data = []
start_all = []
end_all = []
word_all = []

def speech_to_text(config, audio):
    client = speech.SpeechClient()
    operation = client.long_running_recognize(config=config, audio=audio)
    response = operation.result(timeout=90)
    return print_sentences(response)

def print_sentences(response):
    for result in response.results:
        best_alternative = result.alternatives[0]
        transcript = best_alternative.transcript
        confidence = best_alternative.confidence
        data.append(transcript)
        print("-" * 80)
        print(f"Transcript: {transcript}")
        print(f"Confidence: {confidence:.0%}")
        print_word_offsets(best_alternative)
        
def print_word_offsets(alternative):
    start,end,words = [],[],[]
    for word in alternative.words:
        start_s = word.start_time.total_seconds()
        start.append(start_s)
        end_s = word.end_time.total_seconds()
        end.append(end_s)
        word = word.word
        words.append(word)
        print(f"{start_s:>7.3f} | {end_s:>7.3f} | {word}")
        
        start_all.append(start_s)
        end_all.append(end_s)
        word_all.append(word)
    trans = ''.join(data)
    resultdict = {'transcript':trans,'values':{'start':start_all,'end':end_all,'word':word_all}}
    return resultdict
speech_to_text(config, audio)

--------------------------------------------------------------------------------
Transcript: Hello, you have called virtual bank. This is Linda speaking. How may I help you? Hi Linda. I was just at your bill branch and I think I left my debit card in the ATM machine. Okay. Do you have your debit card number? I don't know. Okay. Well, do you have the checking account number associated with the debit card, but I do have are you ready? I will give you what I have got 760-545-6789. Okay. That's +765-450-600-7089. Correct? What is your identification number? 774-589-6589 665 okay. I have +774-580-960-5896 65 and what is your name sir? It is Robert. Appel board. Okay.
Confidence: 72%
  0.000 |   0.400 | Hello,
  0.400 |   1.200 | you
  1.200 |   1.300 | have
  1.300 |   1.800 | called
  1.800 |   2.200 | virtual
  2.200 |   2.400 | bank.
  2.400 |   3.200 | This
  3.200 |   3.400 | is
  3.400 |   3.800 | Linda
  3.800 |   4.300 | speaking.
  4.300 |   5.300 | How
  5.300 |   5.300 | may
  5.

In [10]:
trans = ''.join(data)
resultdict = {'transcript':trans,'values':{'word':word_all, 'start':start_all,'end':end_all}}
print(resultdict)

{'transcript': "Hello, you have called virtual bank. This is Linda speaking. How may I help you? Hi Linda. I was just at your bill branch and I think I left my debit card in the ATM machine. Okay. Do you have your debit card number? I don't know. Okay. Well, do you have the checking account number associated with the debit card, but I do have are you ready? I will give you what I have got 760-545-6789. Okay. That's +765-450-600-7089. Correct? What is your identification number? 774-589-6589 665 okay. I have +774-580-960-5896 65 and what is your name sir? It is Robert. Appel board. Okay.I have Robert Applebaum yet. And what is your date of birth Mr. Appelbaum, July 7th, 1974. Okay, July 7th, 1974. Yes, and your phone number. It is 610-265-1715. Okay, I have 610-265-1715. Yes. Okay, Mr. Appelbaum. I have just this pended your card. If it is in the machine, we will contact you as lift the suspension 00. Thank you, sir. Thank you.", 'values': {'word': ['Hello,', 'you', 'have', 'called', 'v

In [7]:
resultdict2 = {'word':word_all, 'start_times':start_all, 'end_times':end_all}
df = pd.DataFrame.from_dict(resultdict2)
df

,word,start_times,end_times
0,"Hello,",0.0,0.4
1,you,0.4,1.2
2,have,1.2,1.3
3,called,1.3,1.8
4,virtual,1.8,2.2
...,...,...,...
164,Thank,91.7,92.4
165,"you,",92.4,92.5
166,sir.,92.5,93.4
167,Thank,93.4,94.5


In [234]:
dict_ = resultdict['transcript']
dict_

"Hello, you have called virtual bank. This is Linda speaking. How may I help you? Hi Linda. I was just at your bill branch and I think I left my debit card in the ATM machine. Okay. Do you have your debit card number? I don't know. Okay. Well, do you have the checking account number associated with the debit card, but I do have are you ready? I will give you what I have got 760-545-6789. Okay. That's +765-450-600-7089. Correct? What is your identification number? 774-589-6589 665 okay. I have +774-580-960-5896 65 and what is your name sir? It is Robert. Appel board. Okay.I have Robert Applebaum yet. And what is your date of birth Mr. Appelbaum, July 7th, 1974. Okay, July 7th, 1974. Yes, and your phone number. It is 610-265-1715. Okay, I have 610-265-1715. Yes. Okay, Mr. Appelbaum. I have just this pended your card. If it is in the machine, we will contact you as lift the suspension 00. Thank you, sir. Thank you."

In [231]:
# replace . and a space with only a space.
def clean_text(dict_):
    # replace . and a space with only a space.
    text = dict_.replace(".", " ").replace(",", "").replace("\'", "").replace("’", "").replace("-", "").replace("+", "")
    # get rid of the . at the end of each line. 
    cleaned_text = re.sub("\.$","", text)
    
    return cleaned_text

In [232]:
clean_text(dict_)

'Hello you have called virtual bank  This is Linda speaking  How may I help you? Hi Linda  I was just at your bill branch and I think I left my debit card in the ATM machine  Okay  Do you have your debit card number? I dont know  Okay  Well do you have the checking account number associated with the debit card but I do have are you ready? I will give you what I have got 7605456789  Okay  Thats 7654506007089  Correct? What is your identification number? 7745896589 665 okay  I have 7745809605896 65 and what is your name sir? It is Robert  Appel board  Okay I have Robert Applebaum yet  And what is your date of birth Mr  Appelbaum July 7th 1974  Okay July 7th 1974  Yes and your phone number  It is 6102651715  Okay I have 6102651715  Yes  Okay Mr  Appelbaum  I have just this pended your card  If it is in the machine we will contact you as lift the suspension 00  Thank you sir  Thank you '

In [235]:
ori_text = open('data/Text files/Conversation 2.txt').read()
ori_text

'Hello, you have called virtual bank, this is Linda speaking. How may I help you? Hi Linda. I was just at your Ville branch and I think I left my Debit card in the ATM machine. Okay. Do you have your Debit card number? I don’t have. Okay, well do you have the checking account number associated with the Debit card? That I do have. Are you ready? I will give you what I have got. 765-456-789. Okay. That’s 765-456-789. Correct. What is your identification number? 774-589-658-9665. Okay, I have 774-589-658-9665 and what is your name sir? It is Robert Applebaum. Okay. I have Robert Applebaum. Yes. And what is your date of birth Mr. Applebaum? July 7th, 1974. Okay. July 7th, 1974. Yes. And your phone number? It is 610-265-1715. Okay. I have 610-2651715. Yes. Okay Mr. Applebaum. I have just suspended your card. If it is in the machine, we will contact you and lift the suspension. Oh, thank you, Sure. Thank you.'

In [226]:
# replace . and a space with only a space, then make all words lower case.
def clean_text1(ori_text):
    # replace . and a space with only a space, then make all words lower case.
    text = ori_text.replace(".", " ").replace(",", "").replace("\'", "").replace("’", "").replace("-", "").replace("+", "")
    # get rid of the . at the end of each line. 
    cleaned_text = re.sub("\.$","", text)
    
    return cleaned_text

In [228]:
clean_text1(ori_text)

'Hello you have called virtual bank this is Linda speaking  How may I help you? Hi Linda  I was just at your Ville branch and I think I left my Debit card in the ATM machine  Okay  Do you have your Debit card number? I dont have  Okay well do you have the checking account number associated with the Debit card? That I do have  Are you ready? I will give you what I have got  765456789  Okay  Thats 765456789  Correct  What is your identification number? 7745896589665  Okay I have 7745896589665 and what is your name sir? It is Robert Applebaum  Okay  I have Robert Applebaum  Yes  And what is your date of birth Mr  Applebaum? July 7th 1974  Okay  July 7th 1974  Yes  And your phone number? It is 6102651715  Okay  I have 6102651715  Yes  Okay Mr  Applebaum  I have just suspended your card  If it is in the machine we will contact you and lift the suspension  Oh thank you Sure  Thank you '

In [189]:
def Jaccard_Similarity(doc1, doc2): 
    
    words_doc1 = set(doc1.lower().split()) 
    words_doc2 = set(doc2.lower().split())
    
    intersection = set(words_doc1).intersection(set(words_doc2))
    union = words_doc1.union(words_doc2)
        
    return float(len(intersection)) / len(union)

In [233]:
acc = Jaccard_Similarity(clean_text(dict_)/
    , clean_text1(ori_text))
acc = acc*100

print('Accuracy of the conversation:', '%.2f' %acc, '%')

Accuracy of the conversation: 71.43 %


In [9]:
import json
with open('data/GG_Speech/conversation 2.json', 'w') as outfile:
    json.dump(resultdict, outfile)